# Saudi E-Commerce RAG Agent

This notebook implements a Retrieval-Augmented Generation (RAG) agent specifically designed for Saudi e-commerce data. The goal is to enable intelligent question–answering and insights generation using product descriptions, metadata, and domain knowledge.

**Features:**
- Embedding-based document indexing  
- Vector similarity search  
- LLM-based question answering  
- RAG pipeline for context-grounded responses  

**Technologies used:**
- Python
- FAISS / Chroma / (your vector store)
- OpenAI / Llama / other model
- LangChain / LlamaIndex (if applicable)

---


# Install & Import Libraries

In [ ]:
!pip install langchain chromadb sentence-transformers transformers pypdf


In [ ]:
import os

os.listdir()


['.config',
 '8340436664-publichedpaper.pdf',
 "V5.0 Monsha'at SMEM Report - Q2-25.pdf",
 'monsha’at_e_commerce_thematic_report_en.pdf',
 '.gradio',
 '3-Presentation-by-Kingdom-of-Saudi-Arabia.pdf',
 'sample_data']

In [ ]:
!pip install langchain-community


In [ ]:
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
files = [
    "8340436664-publichedpaper.pdf",
    "V5.0 Monsha'at SMEM Report - Q2-25.pdf",
    "monsha’at_e_commerce_thematic_report_en.pdf",
    "3-Presentation-by-Kingdom-of-Saudi-Arabia.pdf"
]

documents = []

for file in files:
    loader = PyPDFLoader(file)
    docs = loader.load()
    documents.extend(docs)

len(documents)


94

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


## Embedding Model and Vector Store Construction

In this section, we initialize the embedding model and build the vector database that powers semantic search.

### **HuggingFace Embeddings**
We use the `all-MiniLM-L6-v2` model from SentenceTransformers, a lightweight and fast model suitable for semantic similarity tasks. This model converts each text chunk into a dense numerical vector (embedding).

### **Chroma Vector Store**
Chroma is used as the vector database to store embeddings and enable efficient similarity search.  
Each product description chunk is embedded and stored inside Chroma for retrieval.

### **Retriever**
After building the vector store, we convert it into a retriever. The retriever will:
- Accept a user query
- Embed the query
- Find the top similar chunks from the vectorstore
- Return them to the RAG agent as context


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Recreate embedding model using the correct wrapper
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Create Chroma vectorstore
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model
)

# Create retriever
retriever = vectorstore.as_retriever()



In [ ]:
from transformers import pipeline

llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_new_tokens=256
)


Device set to use cpu


## 5. Language Model (LLM) Initialization

In this step, we load a lightweight open-source language model to generate natural-language responses based on the retrieved context.

### **FLAN-T5 (Base Model)**
We use the `google/flan-t5-base` model via HuggingFace's `pipeline`.  
FLAN-T5 is a fine-tuned version of T5 that excels in:
- instruction following  
- text generation  
- question answering  
- summarization  

This model is efficient and works well for RAG systems where the heavy lifting of retrieval is handled by the vector store.

### **Pipeline Setup**
The `text2text-generation` pipeline allows us to send prompts and receive generated answers.  
`max_new_tokens=256` controls the maximum length of the model's output.


In [ ]:
#second trail#
from transformers import pipeline

llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256
)




model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
!pip install tavily-python


In [ ]:
from tavily import TavilyClient

# You MUST set the key before creating TavilyClient
os.environ["TAVILY_API_KEY"] = " "

tavily = TavilyClient()

def internet_search(query):
    result = tavily.search(query)
    return result["results"]



### Agent Logic: Combine RAG + Internet Search

This function acts as the main agent responsible for answering user questions.  
It follows a hybrid approach:

1. **Try RAG first** — If the answer exists in the embedded documents, return it immediately.  
2. **Fallback to Web Search** — If RAG cannot answer (e.g., "I don't know"), perform an online search.  
3. **LLM Reasoning** — The retrieved internet results are passed to the LLM to generate a final, clean answer.  

This ensures that the agent always provides the most relevant information available.


In [ ]:
def agent(question):
    # Step 1: Try RAG first
    rag_response = rag_answer(question)

    if "I don't know" not in rag_response:
        return "📘 Answer from PDF:\n\n" + rag_response

    # Step 2: If RAG fails → use internet search
    web_results = internet_search(question)

    combined = ""
    for item in web_results[:3]:  # take top 3
        combined += item["title"] + "\n" + item["content"] + "\n\n"

    # Step 3: Ask LLM using the internet info
    prompt = f'''
Use ONLY the following internet search information to answer the question:

{combined}

Question: {question}

Answer clearly in 4–6 sentences.
    '''
    final = llm(prompt)[0]["generated_text"]

    return "🌐 Answer from Internet:\n\n" + final


In [ ]:
print(agent("What are the top e-commerce platforms in Saudi Arabia?"))


📘 Answer from PDF:

e-commerce is booming. But there’s still room for growth. Compared to leading markets, where online sales account for 18% of retail, 10 e-commerce is relatively untapped in the Kingdom.


### RAG Answer Function (Primary Retrieval-Based Response)

This function handles the Retrieval-Augmented Generation (RAG) workflow.  
It attempts to answer the user's question *only* using information found in the embedded documents.

**Steps performed:**

1. **Retrieve Relevant Chunks**  
   The retriever fetches the top matching text chunks based on semantic similarity.

2. **Normalize Retriever Output**  
   Ensures consistent formatting across different retriever APIs.

3. **Clean and Deduplicate Context**  
   Removes repeated lines and builds a concise context for the LLM.

4. **Construct a Controlled Prompt**  
   The LLM is instructed to answer *strictly* from the retrieved documents and respond  
   `"I don't know based on the provided documents."` if the answer is missing.

5. **Generate the Final RAG Answer**  
   The LLM produces a clear, short 4–6 sentence response grounded in the context.

This function is the core of the RAG pipeline, providing reliable, document-based answers before the system attempts web search fallback.


In [ ]:
def rag_answer(question, k=4):
    # 1. Retrieve documents (new API uses .invoke)
    docs = retriever.invoke(question)

    # Normalize retriever output
    if isinstance(docs, dict):
        docs = docs.get("documents", [])
    elif not isinstance(docs, list):
        docs = [docs]

    # If no relevant documents found
    if not docs:
        return "I don't know based on the provided documents."

    # 2. Build context (clean and remove duplicate lines)
    raw_context = "\n\n".join(d.page_content for d in docs)

    # Remove repeated lines
    lines = raw_context.split("\n")
    unique_lines = list(dict.fromkeys([line.strip() for line in lines if line.strip() != ""]))
    context = "\n".join(unique_lines)

    # 3. Build the prompt for the LLM
    prompt = f"""
You are an AI assistant that answers questions about e-commerce and SMEs in Saudi Arabia.
Use ONLY the information provided in the context below.
If the answer is not in the context, say: "I don't know based on the provided documents."

Context:
{context}

Question: {question}

Provide a clear, short answer in 4–6 sentences without repeating the same lines.
"""

    # 4. Generate the answer
    result = llm(prompt)[0]["generated_text"]

    return result.strip()


In [ ]:
print(rag_answer("What are the main challenges faced by SMEs when adopting e-commerce in Saudi Arabia?"))


Despite its rapid growth over the past five years, the e-commerce sector in Saudi Arabia still faces specific market challenges: While there is no one size fits all solution for many challenges facing SMEs, a number of tried and true solutions can reduce costs, maximize efficiencies, and increase market share.17 Solutions for SMEs in e-commerce The relative youth of Saudi Arabia’s e-commerce market A glance at key developments and statistics As the second-highest venture capital-funded sector in Saudi Arabia, e-commerce is booming. But there’s still room for growth. Compared to leading markets, where online sales account for 18% of retail, 10 e-commerce is relatively untapped in the Kingdom. The growth of e-commerce in the Kingdom 34.5 mn e-commerce users by 20252 42,900 online stores accessible in Saudi Arabia5 26,344 verified e-commerce freelance certificates


In [ ]:
print(rag_answer("How does e-commerce support SME growth in Saudi Arabia?"))


Compared to leading markets, where online sales account for 18% of retail, 10 e-commerce is relatively untapped in the Kingdom. The growth of e-commerce in the Kingdom 34.5 mn e-commerce users by 20252 42,900 online stores accessible in Saudi Arabia5 26,344 verified e-commerce freelance certificates and shopping experience. The dynamic change in demographic, economic, and social factors has become the driver of establishing the growth of e -commerce platforms in Saudi Arabia , which shows that the e -commerce market is projected to achieve the revenue of $14.38bn in 2024, and the revenue is expected to show the annual growth rate (CAGR 2024-2029) of 11% to reach a market volume of $24.30bn in 2029 , as it shown from Statista analysis (Statista, 2024). E-commerce challenges specific to Saudi Arabia


## Building the Gradio User Interface


In [ ]:
import gradio as gr

def chat_fn(question):
    if not question.strip():
        return "Please enter a question."
    answer = rag_answer(question)
    return answer

# Custom CSS for professional styling
custom_css = """
.gradio-container {
    font-family: 'Inter', 'Segoe UI', sans-serif;
    max-width: 1200px;
    margin: 0 auto;
}

.header-container {
    background: linear-gradient(135deg, #1a5f3f 0%, #2d8659 100%);
    padding: 2.5rem 2rem;
    border-radius: 12px;
    margin-bottom: 2rem;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}

.header-title {
    color: white;
    font-size: 2rem;
    font-weight: 700;
    margin-bottom: 0.5rem;
    display: flex;
    align-items: center;
    gap: 0.75rem;
}

.header-subtitle {
    color: rgba(255, 255, 255, 0.9);
    font-size: 1rem;
    font-weight: 400;
}

#question-box, #answer-box {
    border-radius: 8px;
    border: 1px solid #e0e0e0;
    font-size: 0.95rem;
}

#question-box:focus, #answer-box:focus {
    border-color: #2d8659;
    box-shadow: 0 0 0 3px rgba(45, 134, 89, 0.1);
}

.submit-btn {
    background: linear-gradient(135deg, #1a5f3f 0%, #2d8659 100%);
    color: white;
    border: none;
    padding: 0.75rem 2rem;
    border-radius: 8px;
    font-weight: 600;
    font-size: 1rem;
    cursor: pointer;
    transition: all 0.3s ease;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

.submit-btn:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.15);
}

.info-card {
    background: #f8f9fa;
    border-left: 4px solid #2d8659;
    padding: 1rem 1.5rem;
    border-radius: 8px;
    margin: 1.5rem 0;
}

.footer {
    text-align: center;
    color: #666;
    font-size: 0.875rem;
    margin-top: 2rem;
    padding-top: 1.5rem;
    border-top: 1px solid #e0e0e0;
}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    # Header
    gr.HTML("""
        <div class="header-container">
            <div class="header-title">
                 Saudi E-Commerce Insights AI Agent
            </div>
            <div class="header-subtitle">
                Powered by Monsha'at and Saudi E-Commerce Reports
            </div>
        </div>
    """)

    # Info card
    gr.HTML("""
        <div class="info-card">
            <strong>💡 How to use:</strong> Ask questions about Saudi e-commerce trends, market insights,
            regulations, statistics, or any data from the official reports.
        </div>
    """)

    # Main interface
    with gr.Row():
        with gr.Column(scale=1):
            query = gr.Textbox(
                label="Your Question",
                placeholder="e.g., What are the main e-commerce trends in Saudi Arabia?",
                lines=3,
                elem_id="question-box"
            )

            submit = gr.Button(
                "🔍 Get Answer",
                variant="primary",
                elem_classes="submit-btn"
            )

    with gr.Row():
        with gr.Column(scale=1):
            output = gr.Textbox(
                label="Answer",
                lines=10,
                elem_id="answer-box",
                show_copy_button=True
            )

    # Examples section
    gr.Markdown("### 📝 Example Questions")
    gr.Examples(
        examples=[
            "What is the size of the Saudi e-commerce market?",
            "What are the key challenges facing e-commerce in Saudi Arabia?",
            "What regulations govern e-commerce in Saudi Arabia?",
            "What are the growth projections for Saudi e-commerce?"
        ],
        inputs=query
    )

    # Footer
    gr.HTML("""
        <div class="footer">
            <p>© 2024 Saudi E-Commerce Insights | Data sourced from official reports</p>
        </div>
    """)

    # Event handlers
    submit.click(chat_fn, inputs=query, outputs=output)
    query.submit(chat_fn, inputs=query, outputs=output)

demo.launch(share=False, show_error=True)

/tmp/ipython-input-4236657659.py:88: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-4236657659.py:88: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>